In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
def archivo_no_vacio(fpath):  
    return True if os.path.isfile(fpath) and os.path.getsize(fpath) > 0 else False

In [4]:
from collections import defaultdict
empresas_df =  pd.read_csv("constituents_csv.csv", sep=",")
empresas_df["Symbol"] = empresas_df["Symbol"].str.lower()
lista_sectores = list(empresas_df["Sector"])
dic_empresas_industria = defaultdict()
for index, accion in enumerate(empresas_df["Symbol"]):
    dic_empresas_industria[accion] = lista_sectores[index]

In [6]:
# con filtro fecha y filtro 500 empresas
list_files = os.listdir("Stocks1/")
dict_frames = dict() # dicccionario con todos los data frames que rellenaremos
dict_rendimientos = dict()

#recorremos todas las acciones en Stocks desde 2010 (Stocks1)
for nombre_archivo in list_files:
    
    #aplicamos filtro de 500 empresas
    index = nombre_archivo.find(".us")
    simple = nombre_archivo[0:index]
    if simple not in list(empresas_df["Symbol"]):
        continue
        
    path = f"Stocks1/{nombre_archivo}"
    
    # recorremos los archivos no vacios
    if archivo_no_vacio(path):
        
        df =  pd.read_csv(path, sep=",")
        df = df[['Date', 'Open', 'Close', 'Volume']]
        df['ticker'] = nombre_archivo.replace('.csv', '')
        df['Rendimiento'] = (df['Close'] - df['Open']) / df['Open']
        
        # quitamos los datos nulos
        df.dropna(subset=['Rendimiento'], inplace=True)
        
        df['Date']= pd.to_datetime(df['Date'], dayfirst = True)
        
        # filtramos para obtener los datos de hasta el 31 Dic de 2014
        df = df[(df['Date'] < '2015-01-01 00:00:00')]
        
        # se guarda el nombre sin .txt
        dict_frames[nombre_archivo.split(".csv")[0]] = df
        dict_rendimientos[nombre_archivo.split(".csv")[0]] = df[['Rendimiento']]
        
#Veamos una empresa

   

In [52]:
dict_frames['aapl.us']

,Date,Open,Close,Volume,ticker,Rendimiento
0,2010-01-04,27.341,27.406,137045797,aapl.us,0.002377
1,2010-01-05,27.508,27.454,167718021,aapl.us,-0.001963
2,2010-01-06,27.454,27.017,153403690,aapl.us,-0.015918
3,2010-01-07,27.108,26.968,132861904,aapl.us,-0.005165
4,2010-01-08,26.946,27.147,124730848,aapl.us,0.007459
5,2010-01-11,27.274,26.907,128596375,aapl.us,-0.013456
6,2010-01-12,26.806,26.601,165445130,aapl.us,-0.007648
7,2010-01-13,26.652,26.978,168558442,aapl.us,0.012232
8,2010-01-14,26.897,26.819,120618898,aapl.us,-0.002900
9,2010-01-15,27.073,26.370,165632148,aapl.us,-0.025967


In [7]:
# encontremos la maxima presencia bursatil
max_presencias = 0
# hacemos un dict, key = nombre empresa, value = % presencia bursatil
dict_presencias = dict()

# calculo de la maxima presencia bursatil, valor que sera nuestra referencia
#vemos las presencia de todas las acciones y elegimos la mayor
for nombre, df in dict_frames.items():
    # calculamos presencia bursatil de esa accion
    pb = len(df.index)
    if pb > max_presencias:
        max_presencias = pb
        
# guardamos el porcentaje de capitalizacion de cada empresa en el dict
for nombre, df in dict_frames.items():
    # calculamos presencia bursatil de esa accion
    pb_porcentual = len(df.index)/max_presencias
    dict_presencias[nombre] = pb_porcentual
    
dict_presencias
    

{'a.us': 1.0,
 'aal.us': 0.21144674085850557,
 'aap.us': 0.9992050874403816,
 'aapl.us': 1.0,
 'abbv.us': 0.39825119236883944,
 'abc.us': 1.0,
 'abt.us': 1.0,
 'acn.us': 0.9992050874403816,
 'adbe.us': 1.0,
 'adm.us': 1.0,
 'adp.us': 1.0,
 'ads.us': 0.9992050874403816,
 'adsk.us': 1.0,
 'aee.us': 1.0,
 'aep.us': 1.0,
 'aes.us': 1.0,
 'aet.us': 1.0,
 'afl.us': 1.0,
 'aiv.us': 1.0,
 'aiz.us': 0.9992050874403816,
 'ajg.us': 0.9992050874403816,
 'akam.us': 0.9992050874403816,
 'alb.us': 0.9992050874403816,
 'algn.us': 0.9992050874403816,
 'alk.us': 0.9992050874403816,
 'all.us': 1.0,
 'alle.us': 0.21462639109697934,
 'alxn.us': 0.9992050874403816,
 'amat.us': 1.0,
 'amd.us': 1.0,
 'ame.us': 0.9992050874403816,
 'amg.us': 0.9992050874403816,
 'amgn.us': 1.0,
 'amp.us': 0.9992050874403816,
 'amt.us': 0.9992050874403816,
 'amzn.us': 1.0,
 'andv.us': 0.9992050874403816,
 'anss.us': 0.9992050874403816,
 'antm.us': 1.0,
 'aon.us': 0.9992050874403816,
 'aos.us': 0.9992050874403816,
 'apa.us': 1.0

In [8]:
# filtramos con aquellas empresas que cumplen con el 99,9% de las presencias
# creamos un nuevo dict para evitar cambios del dict en la iteracion
dict_frames_nuevo = dict_frames.copy()
no_sirven = 0
for accion, df in dict_frames.items():
    # si tiene menos del 99,9% de presencias se elimina
    if dict_presencias[accion] < 0.999:
        del dict_frames_nuevo[accion]


# actualizamos las acciones
dict_frames = dict_frames_nuevo

#imprimimos la cantidad de acciones
print(len(dict_frames))

443


In [69]:
# hacemos el calculo de matriz de covarianza por industria con lo que tenemos por mientras
# para los datos iniciales
Empresa = namedtuple('Empresa', 'nombre industria sector marketcap')
originales_df =  pd.read_csv("companies.csv", sep=",")
originales_df["ticker"] = originales_df["ticker"].str.lower()
originales_df = originales_df[['ticker', 'company name', 'industry','sector', 'market cap']]
lista_tickers_original = list(originales_df['ticker'])
lista_sectores_original = list(originales_df['sector'])


#dict que guardara las matrices de covarianza por sector
dict_matrices = dict()
for sector in set(lista_sectores_original):
    df_rendimientos = pd.DataFrame()  # data frame vacio que iremos rellenando
    for accion, df in dict_frames_nuevo.items():
        if sector != df['Sector'][0]:
            continue
        else: 
            df_rendimientos[accion] = df['Rendimiento']
    # calculamos la matriz        
    matriz_cov = df_rendimientos.cov()
    

In [10]:
# calculo de la gran matriz de covarianza completa
# crearemos un nuevo data frame que tiene en cada columna los rendimientos de una empresa
# la matriz de covarianza se calcula a partir de este dataframe

df_rendimientos = pd.DataFrame()  # data frame vacio que iremos rellenando

for accion, df in dict_frames.items():
    
    df_rendimientos[accion] = df['Rendimiento']
    
matriz_cov = df_rendimientos.cov()
# guardamos un excel con la matriz
path = f'S&P_cov.csv'
matriz_cov.to_csv(path)



In [11]:
# mostramos la matriz
matriz_cov

,a.us,aap.us,aapl.us,abc.us,abt.us,acn.us,adbe.us,adm.us,adp.us,ads.us,...,xec.us,xel.us,xl.us,xlnx.us,xom.us,xray.us,xrx.us,yum.us,zbh.us,zion.us
a.us,0.000269,4.092102e-06,0.000077,0.000075,5.625322e-05,0.000018,0.000114,0.000083,0.000075,0.000042,...,0.000032,3.873889e-05,0.000108,0.000108,0.000072,0.000110,0.000120,0.000084,9.787730e-05,0.000157
aap.us,0.000004,1.798445e-04,0.000010,-0.000002,-3.079738e-06,0.000040,0.000014,0.000004,0.000004,0.000059,...,0.000072,6.744428e-06,0.000014,0.000020,0.000003,0.000003,0.000011,0.000009,3.438678e-06,0.000024
aapl.us,0.000077,9.821261e-06,0.000181,0.000043,2.679538e-05,0.000017,0.000067,0.000041,0.000043,0.000030,...,0.000031,2.081228e-05,0.000059,0.000067,0.000041,0.000050,0.000075,0.000056,4.946035e-05,0.000076
abc.us,0.000075,-1.733067e-06,0.000043,0.000124,3.468149e-05,0.000005,0.000054,0.000047,0.000041,0.000016,...,0.000024,2.374623e-05,0.000057,0.000045,0.000033,0.000052,0.000055,0.000046,5.353187e-05,0.000057
abt.us,0.000056,-3.079738e-06,0.000027,0.000035,8.377920e-05,0.000007,0.000039,0.000041,0.000036,0.000010,...,0.000006,2.757809e-05,0.000038,0.000032,0.000032,0.000039,0.000046,0.000034,4.394764e-05,0.000054
acn.us,0.000018,4.042271e-05,0.000017,0.000005,6.812486e-06,0.000127,0.000018,0.000010,0.000010,0.000061,...,0.000069,4.343268e-06,0.000014,0.000021,0.000010,0.000009,0.000016,0.000009,7.341917e-06,0.000022
adbe.us,0.000114,1.422859e-05,0.000067,0.000054,3.862933e-05,0.000018,0.000204,0.000061,0.000063,0.000035,...,0.000032,3.030022e-05,0.000082,0.000091,0.000052,0.000077,0.000093,0.000067,6.527100e-05,0.000105
adm.us,0.000083,4.121978e-06,0.000041,0.000047,4.138653e-05,0.000010,0.000061,0.000159,0.000051,0.000023,...,0.000015,3.584133e-05,0.000076,0.000056,0.000052,0.000059,0.000069,0.000051,5.306541e-05,0.000090
adp.us,0.000075,3.814143e-06,0.000043,0.000041,3.592096e-05,0.000010,0.000063,0.000051,0.000080,0.000021,...,0.000016,3.017005e-05,0.000057,0.000058,0.000042,0.000056,0.000064,0.000048,4.864737e-05,0.000076
ads.us,0.000042,5.877450e-05,0.000030,0.000016,9.786956e-06,0.000061,0.000035,0.000023,0.000021,0.000214,...,0.000095,1.387177e-05,0.000035,0.000042,0.000022,0.000032,0.000027,0.000023,2.223300e-05,0.000053
